In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
from scipy import stats
import pandas as pd
import requests
from config import api_key
import numpy as np
from ratelimiter import RateLimiter
from citipy import citipy
from scipy.stats import linregress
import gmaps

# Import API key
from config import gkey

#Access maps with unique API key
gmaps.configure(api_key=gkey)

city_weather_df = pd.read_csv("../WeatherPy/Resources/cities.csv")
city_weather_df


,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Sanguéya,10.7000,-14.3667,67.14,89,22,3.24,GN,1613190637
1,Arraial do Cabo,-22.9661,-42.0278,75.58,88,100,13.91,BR,1613190637
2,Djambala,-2.5447,14.7533,69.89,90,89,3.78,CG,1613190637
3,Butaritari,3.0707,172.7902,81.52,82,3,11.07,KI,1613190637
4,Te Anau,-45.4167,167.7167,65.66,42,64,7.96,NZ,1613190637
...,...,...,...,...,...,...,...,...,...
478,Acopiara,-6.0953,-39.4525,71.74,97,29,3.38,BR,1613191125
479,Tierra Colorada,17.1667,-99.5833,66.88,82,3,3.18,MX,1613191125
480,Pochutla,15.7432,-96.4661,75.99,79,10,3.00,MX,1613191125
481,Quelimane,-17.8786,36.8883,78.80,89,40,4.61,MZ,1613191126


In [2]:
#Use the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = city_weather_df["Humidity"]

#Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, 
                                 max_intensity=np.max(humidity), point_radius=2)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [3]:
#Narrow down the cities to fit weather conditions
#Drop any rows will null values
smaller_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & (city_weather_df["Clouds"]== 0) & \
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
smaller_city_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
141,Hermanus,-34.4187,19.2345,72.00,81,0,8.01,ZA,1613190347
154,Dwārka,22.2394,68.9678,75.40,49,0,5.77,IN,1613190765
208,Harper,4.3750,-7.7169,78.80,89,0,6.62,LR,1613190825
238,Tura,25.5198,90.2201,77.81,33,0,1.79,IN,1613190402
248,Ahipara,-35.1667,173.1667,71.04,59,0,3.69,NZ,1613190083
286,Vila Velha,-20.3297,-40.2925,75.20,88,0,3.44,BR,1613190939
342,Andros Town,24.7000,-77.7667,73.40,94,0,9.22,BS,1613190999
385,Nanhai,30.1282,111.7784,70.00,56,0,6.11,CN,1613191007
390,Bonthe,7.5264,-12.5050,76.12,90,0,3.76,SL,1613191058
444,Bosaso,11.2842,49.1816,71.60,83,0,6.91,SO,1613191118


In [4]:
#Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
141,Hermanus,ZA,-34.4187,19.2345,
154,Dwārka,IN,22.2394,68.9678,
208,Harper,LR,4.3750,-7.7169,
238,Tura,IN,25.5198,90.2201,
248,Ahipara,NZ,-35.1667,173.1667,
286,Vila Velha,BR,-20.3297,-40.2925,
342,Andros Town,BS,24.7000,-77.7667,
385,Nanhai,CN,30.1282,111.7784,
390,Bonthe,SL,7.5264,-12.5050,
444,Bosaso,SO,11.2842,49.1816,


In [5]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : gkey}

https://maps.googleapis.com/maps/api/place/nearbysearch/json?

Object `json` not found.


In [6]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
#     print(response)
#     print(results).json()
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No results found, moving to the next city ")             
        print("_______________________________________________")

Retrieving Results for Index 141: Hermanus.
Closest hotel in Hermanus is Misty Waves Boutique Hotel.
Retrieving Results for Index 154: Dwārka.
Closest hotel in Dwārka is Lemon Tree Premier.
Retrieving Results for Index 208: Harper.
Closest hotel in Harper is Grands' Guesthouse.
Retrieving Results for Index 238: Tura.
Closest hotel in Tura is Hotel Polo Orchid.
Retrieving Results for Index 248: Ahipara.
Closest hotel in Ahipara is The Huts.
Retrieving Results for Index 286: Vila Velha.
Closest hotel in Vila Velha is Golden Tulip Porto Vitória.
Retrieving Results for Index 342: Andros Town.
Closest hotel in Andros Town is Dream Villas.
Retrieving Results for Index 385: Nanhai.
Closest hotel in Nanhai is Jincheng International Hotel.
Retrieving Results for Index 390: Bonthe.
Closest hotel in Bonthe is Bonthe Holiday Village.
Retrieving Results for Index 444: Bosaso.
Closest hotel in Bosaso is Nayruus Hotel.


In [7]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
141,Hermanus,ZA,-34.4187,19.2345,Misty Waves Boutique Hotel
154,Dwārka,IN,22.2394,68.9678,Lemon Tree Premier
208,Harper,LR,4.3750,-7.7169,Grands' Guesthouse
238,Tura,IN,25.5198,90.2201,Hotel Polo Orchid
248,Ahipara,NZ,-35.1667,173.1667,The Huts
286,Vila Velha,BR,-20.3297,-40.2925,Golden Tulip Porto Vitória
342,Andros Town,BS,24.7000,-77.7667,Dream Villas
385,Nanhai,CN,30.1282,111.7784,Jincheng International Hotel
390,Bonthe,SL,7.5264,-12.5050,Bonthe Holiday Village
444,Bosaso,SO,11.2842,49.1816,Nayruus Hotel


In [8]:


info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))